In [1]:
#Installing keras-xlnet
!pip install -q keras-xlnet
!pip install tqdm

In [2]:
# Defining Constants
SEQ_LEN = 512
BATCH_SIZE = 1
EPOCHS = 10
LR = 5e-6

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory




import pandas as pd
import numpy as np
import re
import os
from IPython.display import HTML

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.decomposition import PCA

from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras import optimizers


import warnings
warnings.filterwarnings('ignore')
import os


from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from datetime import datetime

# Any results you write to the current directory are saved as output.
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, sub_size=1, dim=512, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.sub_size = sub_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs[0]) / (self.batch_size*self.sub_size)))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indices = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        #list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(indices)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs[0]))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indices):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X1 = np.empty((self.batch_size, self.dim))
        X2 = np.empty((self.batch_size, self.dim))
        X3 = np.zeros((self.batch_size, 1))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, k in enumerate(indices):
            # Store sample
            X1[i,] = self.list_IDs[0][k,]
            X2[i,] = self.list_IDs[1][k,]

            # Store class
            y[i] = self.labels[k]

        return [X1,X2, X3], keras.utils.to_categorical(y, num_classes=2)

Using TensorFlow backend.


In [4]:
import os
from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint

checkpoint_path = '../input/xlnet-weights/cased_l-24_h-1024_a-16/xlnet_cased_L-24_H-1024_A-16/'

tokenizer = Tokenizer(os.path.join(checkpoint_path, 'spiece.model'))
model = load_trained_model_from_checkpoint(
    config_path=os.path.join(checkpoint_path, 'xlnet_config.json'),
    checkpoint_path=os.path.join(checkpoint_path, 'xlnet_model.ckpt'),
    batch_size=BATCH_SIZE,
    memory_len=0,
    target_len=SEQ_LEN,
    in_train_phase=False,
)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
Embed-Token (EmbeddingRet)      [(None, 256, 1024),  32768000    Input-Token[0][0]                
__________________________________________________________________________________________________
Masking (CreateMask)            (None, 256)          0           Input-Token[0][0]                
__________________________________________________________________________________________________
Embed-Token-Masked (RestoreMask (None, 256, 1024)    0           Embed-Token[0][0]                
                                                                 Masking[0][0]                    
__________

In [5]:
model.inputs

[<tf.Tensor 'Input-Token:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'Input-Segment:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'Input-Memory-Length:0' shape=(?, 1) dtype=float32>]

In [6]:
# @title Convert Data to Array
import os
import numpy as np
from tqdm import tqdm
import os
from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint



checkpoint_path = '../input/xlnet-weights/cased_l-24_h-1024_a-16/xlnet_cased_L-24_H-1024_A-16/'


tokenizer = Tokenizer(os.path.join(checkpoint_path, 'spiece.model'))

def load_data(path):
    global tokenizer
    indices, segments, sentiments = [], [], []
    for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in tqdm(os.listdir(folder)):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()
            ids = tokenizer.encode(text)
            ids = ids[:SEQ_LEN-2]
            seg_ids = [0] * (len(ids)+1)
            ids.extend([4,3])
            seg_ids.append(2)
            if len(ids)<SEQ_LEN:
                delta_len = SEQ_LEN - len(ids)
                ids = [0] * delta_len + ids
                seg_ids = [4] * delta_len + seg_ids
            indices.append(ids)
            segments.append(seg_ids)
            sentiments.append(sentiment)
    items = list(zip(indices, segments, sentiments))
    np.random.shuffle(items)
    indices, segments, sentiments = zip(*items)
    indices = np.array(indices)
    segments = np.array(segments)
    return [indices, segments, np.zeros((len(indices), 1))], np.array(sentiments)
  
  
  
train_path = "../input/imdb-movie-reviews-dataset/aclimdb/aclImdb/train/"
test_path = "../input/imdb-movie-reviews-dataset/aclimdb/aclImdb/test/"


train_x, train_y = load_data(train_path)
test_x, test_y = load_data(test_path)

100%|██████████| 12500/12500 [00:29<00:00, 417.36it/s]


In [7]:
type(train_x[1])

numpy.ndarray

In [8]:
!pip install -q keras-bert

In [9]:
# @title Build Custom Model

import keras

from keras_bert import AdamWarmup, calc_train_steps
from keras.layers import Dense, Dropout,Lambda, Reshape

inputs = model.inputs
prev_output = model.get_layer('FeedForward-Normal-24').output
prev_output = Lambda(lambda x: x[:,-1,:], output_shape=(1024,))(prev_output)
dense = Dense(units=256, activation='tanh')(prev_output)
dense = Dropout(0.1)(dense)
outputs = Dense(units=2, activation='softmax')(dense)

decay_steps, warmup_steps = calc_train_steps(
    test_y.shape[0]//2,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
)

model = keras.models.Model(inputs, outputs)

model.compile(
    AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
Embed-Token (EmbeddingRet)      [(None, 256, 1024),  32768000    Input-Token[0][0]                
__________________________________________________________________________________________________
Masking (CreateMask)            (None, 256)          0           Input-Token[0][0]                
__________________________________________________________________________________________________
Embed-Token-Masked (RestoreMask (None, 256, 1024)    0           Embed-Token[0][0]                
                                                                 Masking[0][0]                    
__________

In [11]:
# @title Initialize Variables
import tensorflow as tf
import keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [12]:
val_x, val_y = [test_x[0][:12500,:],test_x[1][:12500,:],test_x[2][:12500,:]], test_y[:12500]
test_split_x, test_split_y = [test_x[0][12500:,:],test_x[1][12500:,:],test_x[2][12500:,:]], test_y[12500:]

In [13]:
from keras.callbacks import ModelCheckpoint

# Generators
training_generator = DataGenerator(train_x, train_y, batch_size=BATCH_SIZE,dim=SEQ_LEN,sub_size=2)
validation_generator = DataGenerator(val_x, val_y, batch_size=BATCH_SIZE,dim=SEQ_LEN,sub_size=1)

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    callbacks=[ModelCheckpoint('model_weights_512.h5',monitor='val_acc', save_best_only=True)],
                    epochs = EPOCHS,
                    use_multiprocessing=True,
                    workers=2, verbose=2)

Epoch 1/10


ValueError: Error when checking input: expected Input-Token to have shape (256,) but got array with shape (512,)

In [14]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
Embed-Token (EmbeddingRet)      [(None, 256, 1024),  32768000    Input-Token[0][0]                
__________________________________________________________________________________________________
Masking (CreateMask)            (None, 256)          0           Input-Token[0][0]                
__________________________________________________________________________________________________
Embed-Token-Masked (RestoreMask (None, 256, 1024)    0           Embed-Token[0][0]                
                                                                 Masking[0][0]                    
__________

In [15]:
test_gen = DataGenerator(test_x, test_y, batch_size=BATCH_SIZE,dim=SEQ_LEN,sub_size=1)
print(model.evaluate_generator(generator=test_gen))

ValueError: Error when checking input: expected Input-Token to have shape (256,) but got array with shape (512,)